In [1]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import itertools
from skimage.morphology import label
from copy import deepcopy as dcopy,copy

In [2]:
top = [5,1,6,1,8,1,22,7,8]
left = [55,1,6,1,24,3,6,7,2]
N = 9
splits = np.array([list(i) for i in itertools.product([0, 1], repeat = N)],dtype=int)
#top = [5312,1,1,5,4]
#left = [444,5,3,123,1]
#N = 5

In [67]:
class Grid():
    def __init__(self):
        self.potential_grids = [[N,np.ones((N,N),dtype=int)*-1,[0,0,N,N],list(range(2,N+1)),[]]] 
        self.splits =  np.array([list(i) for i in itertools.product([0, 1], repeat = N)],dtype=int)
     
    def add_layer(self,grid,rangecoords,lvl,alignment,coords):
        row_start,col_start,row_end, col_end = rangecoords
        new_grid = dcopy(grid)
        new_coords = dcopy(coords)

        if alignment == 0:
            new_grid[row_start:row_end,col_start] =lvl
            new_grid[row_start,col_start:col_end] =lvl
            
            if(not self.check_line(new_grid[:,col_start],top[col_start])):
                return -1,-1,-1,-1
            if(not self.check_line(new_grid[row_start,:],left[row_start])):
                return -1,-1,-1,-1
            new_coords.append([row_start,col_start])
            row_start +=1
            col_start +=1
    
        if alignment == 1:
            new_grid[row_start:row_end,col_start] =lvl
            new_grid[row_end-1,col_start:col_end] =lvl
            
            if(not self.check_line(new_grid[:,col_start],top[col_start])):
                return -1,-1,-1,-1
            if(not self.check_line(new_grid[row_end-1,:],left[row_end-1])):
                return -1,-1,-1,-1
            new_coords.append([row_end-1,col_start])
            row_end -=1
            col_start +=1


        if alignment == 2:
            new_grid[row_start:row_end,col_end-1] =lvl
            new_grid[row_start,col_start:col_end] =lvl
            
            if(not self.check_line(new_grid[:,col_end-1],top[col_end-1])):
                return -1,-1,-1,-1
            if(not self.check_line(new_grid[row_start,:],left[row_start])):
                return -1,-1,-1,-1
            new_coords.append([row_start,col_end-1])
            row_start +=1
            col_end -=1
        
        if alignment == 3:
            new_grid[row_start:row_end,col_end-1] =lvl
            new_grid[row_end-1,col_start:col_end] =lvl
            
            if(not self.check_line(new_grid[:,col_end-1],top[col_end-1])):
                return -1,-1,-1,-1
            if(not self.check_line(new_grid[row_end-1,:],left[row_end-1])):
                return -1,-1,-1 ,-1
            new_coords.append([row_end-1,col_end-1])
            row_end -=1
            col_end -=1
    
        new_rangecoords = [row_start,col_start,row_end,col_end]


        return 1,new_grid,new_rangecoords,new_coords
    
    
                    
    def check_line(self,line,param):
        for split in self.splits:
            test = line * split
            if self.valid_line(test,param):
                return 1
        return 0
    

    def valid_line(self,pos,param):

        if param == 0:
            return True
        
        blocks = []
        block = 0
        inblock = 0
        for p in pos:
            if p != 0:
                if inblock == 0:
                    block += p
                    inblock = 1
                else:
                    block *=10
                    block +=p
            else:
                if inblock == 1:
                    blocks.append(block)
                    block = 0
                    inblock = 0

        if inblock == 1:
            blocks.append(block)

    #if len(blocks) < 2:
        #    return False        
        if np.gcd.reduce(blocks,dtype=int) !=  param:
            return False
        return True
    
    def solve(self):
        solutions = []
        while len(self.potential_grids) > 0:
            l,g,rc,u,c = self.potential_grids.pop(0)
            lvl = dcopy(l)
            grid = dcopy(g)
            rangecoords = dcopy(rc)
            unused = dcopy(u)
            coords = dcopy(c)
            if lvl == 0:
                solutions.append([grid,coords])
            else:
                #print(lvl)
                if lvl == 1:
                    num = 1
                    b,new_grid,new_rangecoords,new_coords = self.add_layer(grid,rangecoords,num,0,coords)
                    if b == 1:
                        self.potential_grids.append([lvl-1,new_grid,new_rangecoords,[],new_coords])
                else:
                    for num in unused:
                        if num < 2*lvl:
                            for alignment in range(4):
                                b,new_grid,new_rangecoords,new_coords = self.add_layer(grid,rangecoords,num,alignment,coords)
                                if b == 1:
                                    self.potential_grids.append([lvl-1,new_grid,new_rangecoords,[n for n in unused if n != num],new_coords])
                            

        return solutions
    


In [68]:
test = Grid()
soln_list = test.solve()

In [72]:
ans_grid = soln_list[0][0]

In [3]:
coords_list = []
for soln in soln_list:
    coords_list.append(np.array(soln[1]))
np.savez('coords.npz',coords = coords_list)

NameError: name 'soln_list' is not defined

In [44]:
lis = []
for soln in soln_list:
    lis.append(np.array(soln[0]))
np.savez('soln.npz',soln = lis)

In [8]:
soln_file = np.load('soln.npz')
coords_file = np.load('coords.npz')
grids_list = soln_file['soln']
coords_list = coords_file['coords']

In [13]:
N = 9
top = [5,1,6,1,8,1,22,7,8]
left = [55,1,6,1,24,3,6,7,2]
splits = np.array([list(i) for i in itertools.product([0, 1], repeat = N)],dtype=int)

def fixed_cells(grid):
    row_gcd = np.ones((N,N),dtype=int)*-1
    col_gcd = np.ones((N,N),dtype=int)*-1
        
    for i in range(N):
        row_gcd[i,:] = fixed_line(grid[i,:],left[i])
    for i in range(N):
        col_gcd[:,i] = fixed_line(grid[:,i],top[i])

    final = np.ones((N,N),dtype=int)*-1
    for i,j in itertools.product(range(N),range(N)):
        options = np.array([row_gcd[i,j],col_gcd[i,j]])
        if (np.any(options==1)) & (np.all(options !=0)):
            final[i,j] = 1
        if (np.any(options==0)) & (np.all(options !=1)):
            final[i,j] = 0
        if (np.any(options==1)) & (np.any(options==0)):
            return 0,final
    final[grid == 1] = 1
    return 1,final
      
def fixed_line(line,param):
    poss_line =[]
    for split in splits:
        test = line * split
        if valid_line(test,param):
            poss_line.append(split)
    poss_array = np.array(poss_line)
    fixed = np.ones(9,dtype=int)*-1
    for i in range(9):
        if np.all(poss_array[:,i]==1):
            fixed[i] = 1
        elif np.all(poss_array[:,i]==0):
            fixed[i] = 0
    return fixed



def valid_line(pos,param):
    blocks = []
    block = 0
    inblock = 0
    for p in pos:
        if p != 0:
            if inblock == 0:
                block += p
                inblock = 1
            else:
                block *=10
                block +=p
        else:
            if inblock == 1:
                blocks.append(block)
                block = 0
                inblock = 0

    if inblock == 1:
        blocks.append(block)
    
    if np.gcd.reduce(blocks,dtype=int) !=  param:
        return False
    return True
        
i = 0
newgrid_list = []
for grid in grids_list:
    if(i % 500 == 0):
        print(i)
    flag,forced_grid = fixed_cells(grid)
    if flag == 1:
        newgrid_list.append([grid,forced_grid,coords_list[i]])
    i = i + 1

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000


In [17]:
lis = []
for soln in newgrid_list:
    lis.append(np.array(soln[0]))
np.savez('newgrids_list.npz',grids = lis)
lis = []
for soln in newgrid_list:
    lis.append(np.array(soln[1]))
np.savez('fixedgrids_list.npz',fixedgrids = lis)
lis = []
for soln in newgrid_list:
    lis.append(np.array(soln[2]))
np.savez('newcoords_list.npz',coords = lis)

In [3]:
newgrids_file = np.load('newgrids_list.npz')
fixedgrids_file = np.load('fixedgrids_list.npz')
newcoords_file = np.load('newcoords_list.npz')
newgrids_list = newgrids_file['grids']
fixedgrids_list = fixedgrids_file['fixedgrids']
newcoords_list = newcoords_file['coords']

In [16]:
test_grid = newgrids_list[300]
fixedgrid = fixedgrids_list[300]
coords = newcoords_list[300]

In [4]:

def twosquare(grid):
    for i,j in itertools.product(range(8),range(8)):
        if (grid[i,j] > 0 and
            grid[i+1,j+1] > 0 and 
            grid[i+1,j] > 0 and 
            grid[i,j+1] > 0):
                return False 
    return True

def grid_check(grid):
    
    #for i in range(9):
    #    if(np.all(grid[i,:] > -1)):
    #        if(not line_check(grid[i,:],left[i])):
    #            return False
    #    if(np.all(grid[:,i] > -1)):
    #        if(not line_check(grid[:,i],top[i])):
    #            return False

    #for num in range(2,10):
    #    if np.sum(grid == num) > num:
    #       return False

    if np.max(label(grid!=0,connectivity=1)) > 1:
        return False

    if not twosquare(grid):
        return False
      
    return True

def line_check(line,fixedline,param):
    
    for i in range(len(line)):
        if((line[i] == 0) and (fixedline[i] == 1)):
            return False
        if((line[i] != 0) and (fixedline[i] == 0)):
            return False
        
    blocks = []
    block = 0
    inblock = 0
    for p in line:
        if p != 0:
            if inblock == 0:
                block += p
                inblock = 1
            else:
                block *=10
                block +=p
        else:
            if inblock == 1:
                blocks.append(block)
                block = 0
                inblock = 0

    if inblock == 1:
        blocks.append(block)
    
    if np.gcd.reduce(blocks,dtype=int) !=  param:
        return False
    return True



In [5]:
N = 9
top = [5,1,6,1,8,1,22,7,8]
left = [55,1,6,1,24,3,6,7,2]
splits = np.array([list(i) for i in itertools.product([0, 1], repeat = N)],dtype=int)

def perm_splits(n,r):
    res = []
    arr = np.array([list(i) for i in itertools.product([0, 1], repeat = n)],dtype=int)
    for lis in arr:
        if(np.sum(lis) == r):
            res.append(lis)
    return res

In [6]:
N = 9

def solve_grid(test_grid,fixedgrid,coords):
    potential_grids = [[np.ones((N,N),dtype=int)*-1,9,[0,0,9,9]]]
    while(len(potential_grids) > 0):
        g,l,rc = potential_grids.pop(0)
        grid = dcopy(g)
        lvl = dcopy(l)
        #print(lvl)
        rangecoords = dcopy(rc)
        if(lvl == 0):
            print(grid)
            continue
        row_start,col_start,row_end,col_end = rangecoords
        coord = coords[N - lvl]
        row = coord[0];col = coord[1]
        num = test_grid[row][col]
        if((row == row_start)and(col == col_start)):
            alignment = 1
        elif ((row == row_end-1)and(col == col_start)):
            alignment = 2
        elif ((row == row_start)and(col == col_end-1)):
            alignment = 3
        else:
            alignment = 4
        ps = perm_splits(2*lvl-1,num)
        for split in ps:
            grid = dcopy(g)
            #print(grid)
            if alignment == 1:
                grid[row_start:row_end,col_start] = split[0:lvl][::-1]*test_grid[row_start:row_end,col_start]
                grid[row_start,col_start:col_end] = split[lvl-1:]*test_grid[row_start,col_start:col_end]
                new_rangecoords = [row_start+1,col_start+1,row_end,col_end]
            elif alignment == 2:
                grid[row_start:row_end,col_start] = split[0:lvl]*test_grid[row_start:row_end,col_start]
                grid[row_end-1,col_start:col_end] = split[lvl-1:]*test_grid[row_end-1,col_start:col_end]
                new_rangecoords = [row_start,col_start+1,row_end-1,col_end]
            elif alignment == 3:
                grid[row_start,col_start:col_end] = split[0:lvl]*test_grid[row_start,col_start:col_end]
                grid[row_start:row_end,col_end-1] = split[lvl-1:]*test_grid[row_start:row_end,col_end-1]
                new_rangecoords = [row_start+1,col_start,row_end,col_end-1]
            else:
                grid[row_start:row_end,col_end-1] = split[0:lvl]*test_grid[row_start:row_end,col_end-1]
                grid[row_end-1,col_start:col_end] = split[lvl-1:][::-1]*test_grid[row_end-1,col_start:col_end]
                new_rangecoords = [row_start,col_start,row_end-1,col_end-1]
            #print(grid)
            if((line_check(grid[:,col],fixedgrid[:,col],top[col])) and (line_check(grid[row,:],fixedgrid[row,:],left[row]))):
                if(grid_check(grid)):
                    potential_grids.append([grid,lvl-1,new_rangecoords])


In [7]:
i = 0
while(i < len(newgrids_list)):
    solve_grid(newgrids_list[i],fixedgrids_list[i],newcoords_list[i])
    if(i % 10 == 0):
        print(i)
    i = i + 1
    

2500
2510
2520
2530
2540
2550
2560
2570
2580
2590
2600
2610
[[5 5 0 0 0 5 5 0 0]
 [0 9 9 9 9 0 9 0 8]
 [0 0 6 0 4 4 4 7 8]
 [0 9 0 4 3 0 0 0 8]
 [0 9 6 0 3 1 2 0 0]
 [0 9 0 0 3 0 2 7 0]
 [0 9 6 6 6 6 0 7 8]
 [0 7 0 7 0 7 0 7 0]
 [5 8 0 8 8 8 0 0 0]]
2620
2630
2640
2650
2660
2670
2680
2690
2700
2710
2720
2730
2740


KeyboardInterrupt: 

In [28]:
len(coords)

9

In [16]:
newgrids_list[-1]

array([[5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 9, 9, 9, 9, 9, 9, 9, 9],
       [5, 9, 6, 1, 3, 2, 4, 7, 8],
       [5, 9, 6, 3, 3, 2, 4, 7, 8],
       [5, 9, 6, 2, 2, 2, 4, 7, 8],
       [5, 9, 6, 4, 4, 4, 4, 7, 8],
       [5, 9, 6, 6, 6, 6, 6, 7, 8],
       [5, 9, 7, 7, 7, 7, 7, 7, 8],
       [5, 9, 8, 8, 8, 8, 8, 8, 8]])